In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import json
import threading
import math

In [19]:
def get_gameids(n, filter_by='topsellers'):
    gameids = []
    #url = f'https://store.steampowered.com/search/?category1=998&filter={filter_by}&page='
    url = f'https://store.steampowered.com/search/?category1=998&page='
    page = 0

    while page*25 < n:
        page += 1
        response = requests.get(url=url+str(page), headers={'User-Agent': 'Mozilla/5.0'})
        print(url+str(page))
        soup = BeautifulSoup(response.text, 'html.parser')
        for row in soup.find_all(class_='search_result_row'):
            print(row['data-ds-appid'])
            gameids.append(row['data-ds-appid'])
            
            
    print(gameids)
    return gameids[:n]

In [3]:
def scrape(game_id, games_list, verbose=False):
    url = "https://store.steampowered.com/app/" + str(game_id) + '/'

    while True:
        r=requests.get(url)
        if r.status_code==200:
            break 
    
    bs = BeautifulSoup(r.text, 'html.parser')
 
    # Title
    try:
        title = bs.find('div',attrs={'id':'appHubAppName'}).contents[0]
    except:
        title = 'null'
    if verbose:
        print("Added " + title)
        
    # Release date
    try:
        release_date = bs.find('div',attrs={'class':'date'}).contents[0]
    except:
        release_date = 'null'
        
    # Last reviews
    try:
        # [x,y,z], x: positive reviews percentage, y: number of reviews, z: in the last z days
        last_reviews = re.findall(r"[-+]?\d*[,.]\d+|\d+", bs.find('span',attrs={'class':'nonresponsive_hidden responsive_reviewdesc'}).contents[0])
    except:
        last_reviews = 'null'
        
    # Total reviews
    try:
        # [x,y], x: positive reviews percentage, y: number of reviews
        total_reviews = re.findall(r"[-+]?\d*[,.]\d+|\d+", bs.find('div',attrs={'class':'user_reviews_summary_bar'}).find('span',attrs={'class':'game_review_summary positive'})['data-tooltip-html'])
    except:
        total_reviews = 'null'
    
    # Price
    try:
        price = re.findall(r"[-+]?\d*[,.]\d+|\d+", bs.find('div',attrs={'class':'game_purchase_price price'}).contents[0])
    except:
        price = 'null'

    # Developers
    try:
        developers = []
        developers_list = bs.find('div',attrs={'id':'developers_list'})
        developers_names = developers_list.findAll('a')
        for developer in developers_names:
            developers.append(developer.contents[0])
    except:
        developers = 'null'
        
    # Tags
    try:
        game_tags = []
        tags = bs.find('div',attrs={'class':'glance_tags popular_tags'}).findAll('a',attrs={'class':'app_tag'})
        for tag in tags:
            game_tags.append(re.sub(r'\s', '', tag.contents[0]))
    except:
        game_tags = 'null'
        
    # Supported languages   
    try:
        laguage_list = bs.find('div',attrs={'id':'languageTable'}).findAll('tr',attrs={'class':''})
        languages = len(laguage_list) - 1
    except:
        languages = 'null'
        
    
    # OS's supported    
    try:
        os_tabs = bs.find('div',attrs={'class':'sysreq_tabs'}).findAll('div',attrs={'class':'sysreq_tab'})
        compatible_os_number = len(os_tabs)
    except:
        try:
            bs.find('div',attrs={'class':'sysreq_contents'})
            compatible_os_number = 1
        except:
            compatible_os_number = 'null'
    
    
    games_list.append({
                'id': game_id,
                'title': title,
                'release_date': release_date,
                'recent_reviews': last_reviews,
                'all_reviews': total_reviews,
                'price': price,
                'developers': developers,
                'number_languages': languages,
                'compatible_os': compatible_os_number,
                'tags': game_tags
    })

In [4]:
def scrape_ids_list(data, start, interval, file="game_ids_list", verbose=False):
    with open(file, "r") as fp:
         ids = json.load(fp)

    for i in range(start, start+interval):
        if verbose:
            print("Fetching game with url number: " + str(ids[i]))
        try:
            scrape(ids[i], data, True)
            if verbose: 
                print("Success")
        except:
            if verbose:
                print("Failed")

In [20]:
n_ids = 15000
ids = get_gameids(n_ids)
with open("game_ids_list", "w") as fp:
    json.dump(ids, fp)
print("Id search complete, " + str(len(ids)) + " ids")

https://store.steampowered.com/search/?category1=998&page=1
730
1593500
1172470
1446780
1144200
570
1174180
1599340
227300
1551360
230410
1506830
1449850
1085660
1172620
271590
252490
1435790
306130
1569040
359550
1203220
289070
39210
739630
https://store.steampowered.com/search/?category1=998&page=2
1466860
346110
1248130
291550
108600
236390
381210
1240440
1644960
582660
747660
1222670
292030
250900
526870
1250410
242760
294100
1286830
648800
1063730
1062090
1549180
588650
1426210
https://store.steampowered.com/search/?category1=998&page=3
578080
1124300
489830
987840
413150
1091500
270880
311210
578650
1206560
594570
4000
1527950
1097150
1142710
1149460
361420
703080
281990
493520
1086940
534380
221100
261550
594650
https://store.steampowered.com/search/?category1=998&page=4
255710
552990
1432050
1563180
548430
284160
518790
394360
444200
1490890
386360
1135690
460930
620980
546560
1282590
239140
1030840
582010
1245620
323190
440
1151640
962130
1244460
https://store.steampowered.com

https://store.steampowered.com/search/?category1=998&page=35
555570
201810
427730
42960
480490
1623880
242550
990630
1345820
10500
1499120
631510
212070
491540
1039880
70
1695840
607890
292120
496240
1696810
10090
637090
447530
427100
https://store.steampowered.com/search/?category1=998&page=36
1012880
1049410
861540
456750
1390350
910490
751780
1167140
466240
333420
373930
939850
7940
55150
10
1238060
1287840
817540
1118240
1430750
33220
1454400
383980
274190
2640
https://store.steampowered.com/search/?category1=998&page=37
595520
1574270
212680
412830
290340
304430
731490
242920
1557740
855740
1237980
206420
1028350
838380
232890
1040230
996770
1136160
814370
978520
1094780
1097840
311310
1225330
287390
https://store.steampowered.com/search/?category1=998&page=38
1769420
788100
638970
1158370
606880
311560
24780
1379130
1056960
673610
1330460
1276390
108200
895870
675010
1252600
774861
586140
346010
964020
1324780
287980
1552350
1046030
1517970
https://store.steampowered.com/search/?

https://store.steampowered.com/search/?category1=998&page=69
1293160
1517340
1472560
423230
1591520
649950
1580150
285330
1111940
1714590
582560
1277930
40990
434030
1419170
410320
350640
1018800
1502970
1257290
1316230
1359120
535230
948740
287630
https://store.steampowered.com/search/?category1=998&page=70
578620
1842730
24960
35140
846030
688130
720250
1483870
1291010
257850
235620
989440
517790
1227690
1615100
1034860
571260
1183940
952040
1331210
1157390
537180
893180
887060
382560
https://store.steampowered.com/search/?category1=998&page=71
411300
1170970
998740
329050
886250
446020
604500
50
15120
658260
918820
574720
16450
758730
1788780
1229580
1292630
522470
317820
889510
351970
396750
1186040
927350
50300
https://store.steampowered.com/search/?category1=998&page=72
481510
757320
252330
1148650
483770
926990
1128000
1456880
588030
1041840
233270
522230
220740
1775550
359840
319910
851100
605230
1147690
981600
1591420
936500
1602010
273110
808910
https://store.steampowered.com

https://store.steampowered.com/search/?category1=998&page=103
35450
312750
559330
815780
987230
45740
19000
695920
1299360
531640
6300
571310
1177660
868520
345180
942970
466170
835860
1194590
1213230
909080
598060
8190
1079830
1518430
https://store.steampowered.com/search/?category1=998&page=104
1062140
311240
1131620
1197570
292240
914020
1183700
889710
615120
378540
348550
1233590
735290
679900
1519790
722060
1103860
208750
511680
981750
785790
1562940
1259800
1801650
1716940
https://store.steampowered.com/search/?category1=998&page=105
1203930
57690
1223910
30
1102130
742300
423490
966320
1157700
1089790
1154840
246840
252610
297860
41500
418180
3700
225260
487720
1389360
1577620
1251460
1131920
567670
13540
https://store.steampowered.com/search/?category1=998&page=106
794860
1142080
552080
597220
1202280
711980
254480
393520
12360
254320
260750
1442820
1620340
1755920
1142500
1571160
17450
2200
1755350
1788120
1169040
569530
250260
1016730
1000760
https://store.steampowered.com/se

https://store.steampowered.com/search/?category1=998&page=137
888790
1509230
541300
1700
1142390
846730
893520
233980
2210
405900
411830
263540
1173200
15130
1062040
1556650
506140
1269950
1140890
2360,2370,2390,9060
34190
38230
1181840
1608550
207350
https://store.steampowered.com/search/?category1=998&page=138
630610
1315200
312780
1209410
249230
1334730
1475460
201290,204880,228600,228601,255910,487280,974650
1517960
546430
253690
379210
791330
336140
882790
889600
1460490
809440
521800
940710
8790
500810
944430
1037130
1436590
https://store.steampowered.com/search/?category1=998&page=139
354430
1218280
1367320
1101670
1084020
261470
292260
867290
904540
1348700
606710
1084480
296830
253840
13250
1137350
296490
1520
495050
952860
977570
551730
665370
1140170
1029550
https://store.steampowered.com/search/?category1=998&page=140
495280
64000
1264970
1549850
599460
850450
544390
1320100
1253390
2820
1469160
351510
741510
1141120
1570010
262000
210770
407810
6850
1097110
1147450
1561660

...

https://store.steampowered.com/search/?category1=998&page=439
342630
960190
263460
1562230
497700
552970
1304420
1852890
444800
285420
471570
1198860
827640
1504220
835520
485120
513230
381750
38170
380660
1339260
419990
851820
553790
658600
https://store.steampowered.com/search/?category1=998&page=440
482300
1232010
422610
446640
429300
929490
367000
50980
1268100
1357610
259940
695980
671650
709710
1240130
307770
1033880
636150
1253680
11590
330180
876500
302830
216090
923900
https://store.steampowered.com/search/?category1=998&page=441
1673190
1328910
23600
1284110
1458590
1384960
894570
1459910
1552130
50970
270570
255870
897380
547850
595730
1281200
724490
843430
684580
439250
1125480
747190
1008830
736860
1176050
https://store.steampowered.com/search/?category1=998&page=442
1086160
858050
226280
216930
1706090
1379020
298280
897690
615670
1292520
1002410
1587090
338960
1680340
248730
106000
719660
333260
1584450
1819800
1278060
758580
290040
260690
1695620
https://store.steampowe

https://store.steampowered.com/search/?category1=998&page=473
453290
539720
388320
509110
264520
731650
1465380
590850
1672840
255070
257120
1087700
34600
1417730
915490
328500
981260
1000280
818730
492150
1219860
771950
216210
320400
522490
https://store.steampowered.com/search/?category1=998&page=474
315430
277630
1230510
924780
1236960
200490
1264670
15520
549170
621020
1780290
1267930
1198070
1455230
42120
439910
1468220
841270
1421020
1011830
428350
252150
1155340
997060
1247170
https://store.steampowered.com/search/?category1=998&page=475
924250
1165860
1052440
789910
400110
1137490
1199180
9800
1009700
562360
1007730
276360
453780
975430
727930
1422180
630870
730620
483970
812810
1725950
1237540
527710
304500
729370
https://store.steampowered.com/search/?category1=998&page=476
1487450
1136930
551450
1275150
803280
696280
646680
809530
569630
1650250
301380
1196500
1179060
1427400
35480
827940
1437820
976120
380920
818940
831770
386840
305490
274010
17710
https://store.steampower

https://store.steampowered.com/search/?category1=998&page=506
488770
214700
326520
1689310
852790
1163500
1403460
718080
342920
236450,246250,246251,246252,246253,246254,246255,246256,246257
1044870
1231940
788130
1148470
287371,287370
415670
1210010
785780
936620
679830
1814710
514180
507060
1714300
1784220
https://store.steampowered.com/search/?category1=998&page=507
778080
284020
848520
347720
1677360
1296700
1138440
279420
588490
1021850
1804760
1479340
1294780
1054750
1493840
1223390
1122750
1477020
1597060
1686850
1244930
547050
594380
1016000
953370
https://store.steampowered.com/search/?category1=998&page=508
1537480
1001040
330370
1595630
292880
573080
510240
1486800
292840
532890
456170
1799160
752330
521750
27800
1197710
1282890
560350
954010
355310
1487420
877870
461340
204630
1268350
https://store.steampowered.com/search/?category1=998&page=509
1690360
573720
990030
1368780
454900
1155660
1054990
1795980
1172050
502770
1074180
345650
1159450
1072590
851110
957080
1093790
5

https://store.steampowered.com/search/?category1=998&page=539
314360
950370
1147960
1451760
1439320
557040
597730
528610
1484580
1567490
1123460
1221860
1243790
1096610
1315160
797410
1536470
792410
1351560
356890
1736900
628950
1222170
334210
772540
https://store.steampowered.com/search/?category1=998&page=540
853110
446790
1786590
927310
1263490
330770
1527520
1620640
1015750
1355310
1603860
2280,2290,2300,9160
1324870
1170180
1204440
1479310
503060
997950
962220
1059740
464780
1025070
329340
1274820
1283210
https://store.steampowered.com/search/?category1=998&page=541
1259130
1066470
1257930
3420
446460
829940
814650
586230
568770
32610
1079970
563710
593240
680700
1345120
612820
658050
831470
1514000
1055690
586320
1745220
788380
942930
394850
https://store.steampowered.com/search/?category1=998&page=542
278590
749010
1307070
250580
1540400
1280380
876190
328270
625780
1165750
466050
1483800
987190
1584540
462730
270770
662240
1633060
974390
1641520
878250
206440,206450
1362650
714

https://store.steampowered.com/search/?category1=998&page=572
1532780
501530
1782880
1113300
543140
1409340
502410
853310
1044480
1506660
800490
1095370
578870
894000
940810
1587350
350480
332370
640890
405720
1853270
1821090
359440
1617450
390910
https://store.steampowered.com/search/?category1=998&page=573
568490
1105410
1494190
377140
535700
1116650
751330
1455090
878170
1467830
331450
643200
1630590
929200
1275860
401690
298790
1579680
1248140
1480050
824760
413580
746620
366100
630460
https://store.steampowered.com/search/?category1=998&page=574
528820
896570
1694100
1002
1267370
514290
1248350
374520
225360
1067400
1676710
263060
1557430
521430
403700
407340
640880
1036490
1661450
1232500
1450310
1712420
402390
447980
1002270
https://store.steampowered.com/search/?category1=998&page=575
921730
302060
992210
1029570
337890
1367250
1800970
729300
903850
1186290
1627420
294710
1062540
502540
682480
16730
586920
831060
1590700
348180
278640
550990
1083450
321150
267900
https://store.

In [ ]:
data = []
threads = []
n_threads = 16
interval = int(math.ceil(n_ids/n_threads))
print(interval)

# start all of the threads
for i in range(0, n_threads):
    thread = threading.Thread(target = scrape_ids_list, args=(data,interval*(i+1),interval,))
    thread.start()
    print("Started thread: "+str(i))
    threads.append(thread)

# now wait for them all to finish
for thread in threads:
    thread.join()

print("Scraping search complete")

938
Started thread: 0
Started thread: 1
Started thread: 2
Started thread: 3
Started thread: 4
Started thread: 5
Started thread: 6
Started thread: 7
Started thread: 8
Started thread: 9
Started thread: 10
Started thread: 11
Started thread: 12
Started thread: 13
Started thread: 14
Started thread: 15
Added Radiant DefenseAdded KartingAdded Love, Sam


Added Maitetsu
Added SATAZIUS
Added Lil Big InvasionAdded ColdSideAdded Half-Life: Blue ShiftAdded 1-Bit Revival: The Residuals of NullAdded Button Button Up!

Added Assassin’s Creed® Chronicles: Russia



Added Fantasy of Expedition
Added A grande bagunça espacial - The big space mess
Added Project Wingman
Added Hidden & Dangerous 2: Courage Under Fire
Added Rinlo
Added Intrusion 2
Added Brothers in Arms: Hell's Highway™
Added Heroes of Normandie
Added Mr.Mine
Added Tower Wars
Added MudRunner
Added SiralimAdded Darkness and Flame: Born of FireAdded Move Code Lines


Added Legend of ManaAdded Space Simulation Toolkit
Added Jack N' Jill DXAdde

Added Archangel™: Hellfire - Enlist FREE
Added Biathlon Battle VRAdded inbento

Added WattamAdded Concept Destruction

Added Darwin ProjectAdded Koihime Enbu RyoRaiRai 恋姫†演武 ～遼来来～

Added Yuppie Psycho: Executive EditionAdded 大城主战略版

Added Runaway: A Twist of Fate
Added Nephise: Ascension
Added In Sound Mind
Added Sword of the Stars II: Enhanced Edition
Added Next 4
Added Dungeon and Puzzles
Added Far from Noise
Added War RoomAdded Why Am I Dead At SeaAdded Bunny's Ban


Added Solitaire VR
Added Fancy Skiing VR
Added Where the Water Tastes Like WineAdded Shadowplay: Metropolis Foe
Added Can't Drive This

Added Hacknet
Added Bejeweled DeluxeAdded Clone Drone in the Danger Zone

Added Three Fairies' Hoppin' Flappin' Great Journey!
Added Extreme Racing on Highway
Added The Immortal Mayor
Added Oneiro
Added Diabolic
Added Bastard BondsAdded Sam & Max Hit the Road
Added KUBOOMAdded SiNiSistar Lite Version


Added SURVIVAL: Postapocalypse NowAdded Strike Suit Infinity

Added HOARD
Added Oh Je

Added Myst IV: RevelationAdded EVEREST VR™

Added The Drift Challenge
Added MahjongAdded Real Boxing™Added Milo and the MagpiesAdded Arm of Revenge Re-Edition



Added The Lar
Added Happy's Humble Burger Farm
Added Sakura Dungeon
Added ABRACA - Imagic Games
Added Shop Tycoon: Prepare your walletAdded VoxelgramAdded 长安夜明


Added I hate this game
Added DYNASTY WARRIORS 8: Xtreme Legends Complete EditionAdded American Fugitive

Added Karma City Police
Added Vampire: The Masquerade — Out for BloodAdded Abda Redeemer: Space alien invasion

Added AKIBA'S TRIP: Hellbound & Debriefed
Added Castlevania Anniversary Collection
Added Country Girl Keiko
Added Seed of the Dead: Sweet HomeAdded Rogue State

Added Simply ChessAdded Airscape - The Fall of Gravity
Added Antonball DeluxeAdded Nursery Slime


Added Ultimate Arena
Added Russian Life Simulator
Added Flight of the Amazon Queen: 25th Anniversary Edition
Added Kathy Rain: Director's Cut
Added Season Match
Added Scarlett Mysteries: Cursed Child

Added COLLAPSEDAdded A Fold Apart

Added Demonheart
Added Northern Tale
Added DUNGEONS OF CHAOS
Added Idol Manager
Added Home Design 3D
Added Harvest Moon: One World
Added 5.0
Added Airplane ModeAdded Fossil Corner

Added Victory At Sea Pacific
Added LOVEAdded TouHou Makuka Sai ~ Fantastic Danmaku Festival Part II
Added ShyChess

Added Trianga's Project: Battle Splash 2.0
Added GlowfishAdded Saviors of Sapphire Wings / Stranger of Sword City Revisited

Added Towaga: Among Shadows
Added Bridge Constructor: The Walking Dead
Added Assassin's Creed® III Remastered
Added Bright Memory
Added Granny Simulator
Added Scream CollectorAdded Solar System VR

Added Punk Wars
Added Parker & Lane: Twisted Minds
Added pear potion🍐
Added BAJA: Edge of Control HD
Added Best Day Ever
Added 100 hidden aliens
Added Type:Rider
Added Trickster VR: Co-op Dungeon Crawler
Added KnightedAdded Swords and Sandals Spartacus

Added DISGAEA RPG
Added Behind the Frame: The Finest Scenery
Added STAR WARS™ Jedi Knight: 

Added Shmups Skill Test シューティング技能検定
Added Dungeon Of Dragon Knight
Added Drawn®: The Painted Tower
Added Megapolis
Added Crisis VRigade
Added Circadian City
Added Gorilla TagAdded Blind

Added Call of War: World War 2
Added Push Me Pull You
Added Putin VS ZombiesAdded Grim Tales: Trace in Time Collector's Edition

Added Lara Croft GO
Added AlveoleAdded Battle Academy 2: Eastern FrontAdded Just Cause 4 Reloaded
Added Westerado: Double Barreled
Added Oafmatch


Added Linelight
Added Roads of Rome: New Generation
Added Space Bob vs. The Replicons
Added The Purring Quest
Added THE KING OF FIGHTERS 2002 UNLIMITED MATCH
Added The Settlers® 7 : History Edition
Added Touhou DollDraft
Added Puppet Fever
Added TY the Tasmanian Tiger
Added L.A. Noire
Added THE ENIGMA MACHINE
Added The Lost Legends of Redwall™: The Scout Act 1Added Phantom Trigger

Added Rune KnightsAdded Time Trap - Hidden Objects Puzzle GameAdded Sea Dogs: To Each His Own - Pirate Open World RPG

Added Hover Junkers

Added A Way

...

Added Zanzarah: The Hidden Portal
Added Finding Paradise
Added Bird path
Added Ragdoll Runners
Added Tanuki Sunset
Added Conan the mighty pigAdded √Letter - Root Letter -

Added Get In The Car, Loser!
Added FPV FreeriderAdded Kingdom Wars

Added Hitman: ContractsAdded OLEG MONGOL

Added Warhammer 40,000: Dawn of War IIAdded SimplePlanes VR

Added Oik Memory 2
Added Raiden V: Director's Cut | 雷電 V Director's Cut | 雷電V:導演剪輯版Added Argonus and the Gods of Stone

Added Quin
Added LABYRINTH OF TOUHOU - GENSOKYO AND THE HEAVEN-PIERCING TREE
Added Soer Dolls
Added The Radio Station | 深夜放送
Added Chicken Invaders 3
Added Citizen of Rome - Dynasty Ascendant
Added In Other Waters
Added Santa's Story of Christmas
Added Broken Armor
Added Realm Revolutions
Added Paper Dolls VRAdded Homebrew - Patent Unknown

Added Notruf 112 | Emergency Call 112Added 3D Ultra™ Minigolf AdventuresAdded The spy who shot me™


Added Paper Dolls 2 纸人贰
Added Vambrace: Cold Soul
Added The Incredible Adventures of Van Hels

Added Behind Glass: Aquarium Simulator
Added Sword and Fairy 4Added Jagged Alliance: Rage!
Added INFERNIUM

Added Hover
Added Lilith Odyssey
Added My Singing Monsters
Added Littlewitch Romanesque: Editio RegiaAdded 8-Bit Adventures 1: The Forgotten Journey Remastered Edition

Added Quake III ArenaAdded Fight of Animals: Arena

Added For EvelynAdded Guards

Added Nuclear Assault
Added Pilot Brothers 2
Added Cabin Fever
Added Dance Collider
Added Strife: Veteran Edition
Added Emily Wants To PlayAdded COPPER ODYSSEY

Added klocki
Added EARTH DEFENSE FORCE 4.1 The Shadow of New Despair
Added Army Men: Toys in SpaceAdded Gedonia

Added PsiloSybilAdded Northmark: Hour of the Wolf

Added TimbermanAdded All My Gods

Added BREAK ARTS II
Added BBQ Simulator: The Squad
Added Battlevoid: Sector Siege
Added Lightfish
Added Front Defense: Heroes
Added 1001 Jigsaw. Legends of MysteryAdded Black Ice

Added The Myth Seekers 2: The Sunken City
Added Blacksmith of the Sand Kingdom
Added Planet Alcatraz 2


Added Squally
Added Through the Darkest of TimesAdded 1 Moment Of Time: Silentville

Added CAYNE
Added SHOGUN: Total War™ - Collection
Added Wayout 2: Hex
Added Yoshiwara Higanbana
Added Graceful Explosion MachineAdded Ghosts of War: Battle Royale WW2 Shooting gamesAdded StonekeepAdded Music Killer



Added Age of Wonders Shadow Magic
Added Phantom Abyss
Added PapeturaAdded Peppy's Adventure

Added TETRA's Escape
Added State of MindAdded My Singing Monsters Playground
Added Card Hunter

Added Achievement Idler: Red
Added Samurai of Hyuga Book 2
Added Alien Breed: Impact
Added Stickit
Added Turok 2: Seeds of Evil
Added BARRICADEZ
Added Caesar™ 3
Added IL-2 Sturmovik: 1946
Added G.I. Joe: Operation Blackout
Added Bobok
Added Everyday Shooter
Added Agatha KnifeAdded Viking Saga: The Cursed Ring
Added Drive 4 Survival

Added Demetrios - The BIG Cynical Adventure
Added STAR WARS™ X-Wing vs TIE Fighter - Balance of Power Campaigns™
Added ALICE VR
Added Math FunAdded Arsenal of Democracy: A 

Added Unicorn DungeonAdded Tom Clancy's Rainbow Six® Vegas 2

Added Legacy - Witch Island 2
Added Spinnortality | cyberpunk management sim
Added Blast Beat
Added The Persistence
Added Haydee
Added Die Young
Added Peachleaf Pirates
Added Labyrinthine Dreams
Added Super House of Dead NinjasAdded NAIR

Added Mu Cartographer
Added Totally Accurate Battlegrounds
Added Flight Control HD
Added DUSK '82: ULTIMATE EDITION
Added Penguin ClimbingAdded BLUE REFLECTION / BLUE REFLECTION　幻に舞う少女の剣

Added B67
Added Neversong
Added Galactic Mining CorpAdded Zup! 4

Added VCB: Why City 4k
Added One Last MemoryAdded F1 RACE STARS™

Added Drawn™: Trail of Shadows Collector's Edition
Added Stealth Bastard DeluxeAdded Rugby League Team Manager 2018

Added Mansions of MadnessAdded The Deer God

Added Lost Lands: The Wanderer
Added Overclocked: A History of Violence
Added 九州：商旅（Nine Provinces: Caravan）
Added Headliner: NoviNews
Added Oniken: Unstoppable Edition
Added Sam & Max 301: The Penal Zone
Added Mayhem

Added Horror SquadAdded COLINA: Legacy

Added Two Brain Sides
Added Late ShiftAdded Phoenix Force

Added Receiver
Added The Eye of Borrack
Added Raging Loop
Added The Legend of Bum-Bo
Added Muddledash
Added Trillion: God of Destruction
Added Bowl Bound College FootballAdded RRRR

Added Shadwen
Added Onimusha: Warlords
Added Marble TrapAdded Monica e a Guarda dos Coelhos

Added Romeow: in the cracked worldAdded Foreskin Fury

Added HAAK
Added Huusuienbu - Chapter spring and Summer
Added Marle: The Labyrinth of the Black Sea
Added Degrees of Separation
Added Autobahn Police Simulator 2
Added Aces High III
Added INVERT
Added Hentai Vs Furries
Added The TapeAdded DOOM 64Added Bloody Raid

Added Highway Blossoms

Added Tangent Rush
Added Space Empires IV Deluxe
Added Little Brother Jim
Added Age of Fear: The Undead King GOLDAdded Space Elite Force

Added Cage of the Succubi
Added WolfQuest: Anniversary Edition
Added Sakura Gamer
Added Kombi Travels - Jigsaw LandscapesAdded Birth ME CodeAdde

In [ ]:
data

In [10]:
df=pd.DataFrame(data)
df = df[["id", "title", "release_date","recent_reviews","all_reviews","price","developers","number_languages","compatible_os","tags"]]

In [3]:
df

,title,release_date,recent_reviews,all_reviews,price,developers,number_languages,compatible_os,tags
0,Radiant Defense,"11 Jul, 2014","['83', '95']","['83', '95']","['4,99']",['Hexage'],7,1,"['Strategy', 'Indie', 'TowerDefense']"
1,Karting,"15 Apr, 2021","['85', '14', '30']","['88', '326']","['1,99']",['RewindApp'],1,1,"['Racing', 'Casual', 'Multiplayer', 'Simulatio..."
2,"Love, Sam","31 May, 2019","['94', '175']","['94', '175']","['2,39']",['Korean Linguistics Lab'],1,1,"['Adventure', 'Indie', 'Violent', 'Horror', 'P..."
3,Maitetsu,"29 Jun, 2018","['95', '285']","['95', '285']","['12,49']",['Lose'],2,1,"['Cute', 'Anime', 'VisualNovel', 'Trains', 'Ca..."
4,SATAZIUS,"16 Dec, 2011","['90', '151']","['90', '151']",NaN,['ASTRO PORT'],2,2,"[""Shoot'EmUp"", 'Indie', 'Action', 'Arcade', '2..."
5,1-Bit Revival: The Residuals of Null,"20 Mar, 2019","['100', '11']","['100', '11']",NaN,['Alex Hanson-White'],18,2,"['Action', 'Indie', 'PixelGraphics', 'Retro', ..."
6,Button Button Up!,"9 Aug, 2019","['98', '82']","['98', '82']","['9,99']",['Centrosphere Games'],4,1,"['Sokoban', '2DPlatformer', 'PuzzlePlatformer'..."
7,Half-Life: Blue Shift,"1 Jun, 2001","['92', '288', '30']","['91', '8,330']","['3,99']",['Gearbox Software'],3,3,"['FPS', 'Action', 'Sci-fi', 'Singleplayer', 'S..."
8,ColdSide,"6 Nov, 2020","['78', '14', '30']","['80', '277']","['6,59']",['AdroVGames'],5,1,"['Adventure', 'Indie', 'Singleplayer', 'Psycho..."
9,Assassin’s Creed® Chronicles: Russia,"9 Feb, 2016","['68', '16', '30']",NaN,"['9,99']",['Climax Studios'],13,1,"['Action', 'Adventure', 'Assassin', 'Stealth',..."


In [4]:
#removing duplicated rows
df.drop_duplicates(subset='title',keep = "first", inplace = True)
df.dropna(subset=['title'])
df

,title,release_date,recent_reviews,all_reviews,price,developers,number_languages,compatible_os,tags
0,Radiant Defense,"11 Jul, 2014","['83', '95']","['83', '95']","['4,99']",['Hexage'],7,1,"['Strategy', 'Indie', 'TowerDefense']"
1,Karting,"15 Apr, 2021","['85', '14', '30']","['88', '326']","['1,99']",['RewindApp'],1,1,"['Racing', 'Casual', 'Multiplayer', 'Simulatio..."
2,"Love, Sam","31 May, 2019","['94', '175']","['94', '175']","['2,39']",['Korean Linguistics Lab'],1,1,"['Adventure', 'Indie', 'Violent', 'Horror', 'P..."
3,Maitetsu,"29 Jun, 2018","['95', '285']","['95', '285']","['12,49']",['Lose'],2,1,"['Cute', 'Anime', 'VisualNovel', 'Trains', 'Ca..."
4,SATAZIUS,"16 Dec, 2011","['90', '151']","['90', '151']",NaN,['ASTRO PORT'],2,2,"[""Shoot'EmUp"", 'Indie', 'Action', 'Arcade', '2..."
5,1-Bit Revival: The Residuals of Null,"20 Mar, 2019","['100', '11']","['100', '11']",NaN,['Alex Hanson-White'],18,2,"['Action', 'Indie', 'PixelGraphics', 'Retro', ..."
6,Button Button Up!,"9 Aug, 2019","['98', '82']","['98', '82']","['9,99']",['Centrosphere Games'],4,1,"['Sokoban', '2DPlatformer', 'PuzzlePlatformer'..."
7,Half-Life: Blue Shift,"1 Jun, 2001","['92', '288', '30']","['91', '8,330']","['3,99']",['Gearbox Software'],3,3,"['FPS', 'Action', 'Sci-fi', 'Singleplayer', 'S..."
8,ColdSide,"6 Nov, 2020","['78', '14', '30']","['80', '277']","['6,59']",['AdroVGames'],5,1,"['Adventure', 'Indie', 'Singleplayer', 'Psycho..."
9,Assassin’s Creed® Chronicles: Russia,"9 Feb, 2016","['68', '16', '30']",NaN,"['9,99']",['Climax Studios'],13,1,"['Action', 'Adventure', 'Assassin', 'Stealth',..."


In [20]:
df.to_csv('steam_data.csv', index=False)